# Multilabel Image Classification_ fastai

In order to predict RLE segments of the steel images in the test set, I've decided to first train a model that predicts whether or not an image has defects. Although, to make it more interesting, my model is going to predict how many defects a steel image has. As you will see later in my analysis, an image can have 0-3 defects. 

The aim of this kernel is to train a **cnn_learner with resnet34 architecture** and save the weights of the model (if I can figure out how to save and reuse the model in another kernel! :P) and also export the predicted labels for the test set so that I can exclude those images in my segmentation solution.

People who inspired the idea are [Mayur Kulkarni](https://www.kaggle.com/mayurkulkarni/fastai-simple-model-0-88-lb) and [xhlulu](https://www.kaggle.com/xhlulu/severstal-simple-2-step-pipeline). Please go and check out their kernels and vote up their kernels if you like their approach. 

Also, thanks to [Jeremy Howard](https://www.kaggle.com/jhoward) for his great deep learning tutorials. I'm following his instructions while still trying to get my head round how to use _fastai_, so bear with me Jeremy! :D    

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# import required libraries
import numpy as np 
import pandas as pd 
from fastai import *
from fastai.vision import *
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing import image
from pathlib import Path
import os
import glob  # used for loading multiple files
# import PIL 
!mkdir -p /tmp/.cache/torch/checkpoints/
# !cp /input/resnet34/resnet34.pth /tmp/.cache/torch/checkpoints/resnet34-333f7ec4.pth

### File operations

In [ ]:
# root directory
path = Path("../input/severstal-steel-defect-detection");
path.ls()

In [ ]:
path_train_img = path/'train_images';
path_test_img = path/'test_images';

### Sneak peek in the train data

In [ ]:
# get a list of filenames in the train image directory
fnames = get_image_files(path_train_img)
fnames[:5]

In [ ]:
# labels in the train.csv file  
train = pd.read_csv(path/'train.csv')
train.head()

**Observations:**

_ClassId_ is attached to the filename. Each filename has 4 classes 1-4 and each class does or does not have a mask (run Length Encoding) associated to it. Mask is represented as NaN when there does not an RLE string for a _ClassId_. An image may have none, one or multiple RLE codes.  

### Visualize images and masks

In [ ]:
# function to plot an image
def plot_img(ImageId):
    img_id = ImageId+'.jpg'
    img = open_image(str(path_train_img) + '/'+img_id)
    return img

# function to plot a mask of an image
def plot_mask(ImageId_ClassId):
    mask = open_mask_rle(train.loc[lambda df: df["ImageId_ClassId"] == ImageId_ClassId, "EncodedPixels"].values[0], shape=(256, 1600))
    mask = ImageSegment(mask.data.transpose(2, 1))
    return mask

def plot_img_mask(ImageId,ClassId):
    defect_img = plot_img(ImageId)
    defect_mask = plot_mask(ImageId+'.jpg_'+str(ClassId))
    defect_img.show(y=defect_mask, figsize=(20, 10), title = 'image & its masks')

In [ ]:
plot_img('0002cc93b')

In [ ]:
plot_mask('0002cc93b.jpg_1')

In [ ]:
plot_img('fff02e9c5')

In [ ]:
plot_mask('fff02e9c5.jpg_3')

In [ ]:
plot_img('000f6bf48')

In [ ]:
plot_mask('000f6bf48.jpg_4')

In [ ]:
# Visualize mask and image in one plot
plot_img_mask('000f6bf48',4)

# Construct Masks from RLE strings

I liked Mayur's idea on pivoting the labels for each class (class1-class4) so that we'll have one line per _ImageId_. I believe this will prevent overfitting when splitting the data into train and validation i.e. no ImageId will leak from train to the validation set.

In [ ]:
# https://www.kaggle.com/mayurkulkarni/fastai-simple-model-0-88-lb
def train_pivot(train_csv):
    df = pd.read_csv(train_csv)

    def group_func(df, i):
        reg = re.compile(r'(.+)_\d$')
        return reg.search(df['ImageId_ClassId'].loc[i]).group(1)

    group = df.groupby(lambda i: group_func(df, i))

    df = group.agg({'EncodedPixels': lambda x: list(x)})

    df['ImageId'] = df.index
    df = df.reset_index(drop=True)

    df[[f'EncodedPixels_{k}' for k in range(1, 5)]] = pd.DataFrame(df['EncodedPixels'].values.tolist())
    
    df = df.drop(columns='EncodedPixels')
    train_df = df.fillna(value=' ')
    return train_df

In [ ]:
train_df = train_pivot(str(path)+'/train.csv')
train_df.head()

In [ ]:
# adding a flag to determine whether or not an image has defects
train_df['has_defects'] = 0
train_df.loc[(train_df['EncodedPixels_1'] != ' ') |  (train_df['EncodedPixels_2'] != ' ') | (train_df['EncodedPixels_3'] != ' ')
            | (train_df['EncodedPixels_4'] != ' '), 'has_defects'] = 1 

In [ ]:
train_df.head()

In [ ]:
print('There are' ,train_df[train_df.has_defects == 1].shape[0] , 'images with defects and' , train_df[train_df.has_defects == 0].shape[0]
      , 'without defects in the training set') 

In [ ]:
# using the original dataframe where there are 4 lines for each ImageId, I calculate the number of defects for each image

tmp = train.copy()
tmp['ImageId'] = tmp['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
tmp['ClassId'] = tmp['ImageId_ClassId'].apply(lambda x: x.split('_')[1])
tmp['has_defects'] = tmp.EncodedPixels.apply(lambda x: 1 if not pd.isnull(x) else 0)
defects = pd.DataFrame(tmp.groupby(by="ImageId")['has_defects'].sum())
defects.reset_index(inplace=True)  # convert the image_id which is an index to a column so that the dataframes can be joined on that
defects.rename(columns={"has_defects": "no_of_defects"},inplace=True) # rename the aggregated column ready for the join 
defects.head()


# add the no_of_defects to the labels dataframe

train_df = train_df.merge(defects, left_on='ImageId', right_on='ImageId', how='left')
train_df.head(4)

In [ ]:
sns.countplot(train_df.no_of_defects)

In [ ]:
train_df.no_of_defects.value_counts().sort_values(ascending=False)

In [ ]:
# example of steel images more than 1 defects
train_df[train_df.no_of_defects > 1]

In [ ]:
# an example image with 2 defects: class 3 & class 4
plot_img('fd26ab9ad')

In [ ]:
plot_img_mask('fd26ab9ad','3')
plot_img_mask('fd26ab9ad','4')

# Model : Image Classifier to predict non-defect images in the test set

I use the number of defects as the dependent variable in my classifier. Here, I create a subset of the train_df dataframe that contains the ImageId and no_of_defects columns to predict number of defects.

In [ ]:
train_clf = train_df[['ImageId','no_of_defects']] 
train_clf.head()

In [ ]:
# creating the specific data format called ImageDataBunch required by the fastai models. It bundles the actual training images
# in the image directory with the labels we loaded into a dataframe 
np.random.seed(42)
bs = 64
data = ImageDataBunch.from_df(path_train_img, train_clf, ds_tfms=get_transforms(), size=256, bs=bs, test=path_test_img
                                  ).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

In [ ]:
print(data.classes)
len(data.classes),data.c

## Train the model with Resnet34

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate, model_dir="/kaggle/working")

In [ ]:
learn.model

In [ ]:
learn.fit_one_cycle(1)

In [ ]:
learn.save('DefectClass_stage-1')

In [ ]:
# !mkdir exports

In [ ]:
# learn.export()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=2)

In [ ]:
learn.recorder.plot()

In [ ]:
# ingest more data into the model to improve error_rate
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=slice(1e-04,1e-03))

In [ ]:
learn.save('DefectClass_stage-2')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.most_confused(min_val=2)

In [ ]:
learn.recorder.plot()

#### Can our model do any better by fine-tunning even more?

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, max_lr=slice(1e-05,1e-04))

In [ ]:
learn.save('DefectClass_stage-3')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.most_confused(min_val=2)

In [ ]:
learn.predict(is_test=True)

In [ ]:
learn.show_results()

# Cleaning up

In [ ]:
from fastai.widgets import *

In [ ]:
ds, idxs = DatasetFormatter().from_toplosses(learn, n_imgs=100)

In [ ]:
ImageCleaner(ds, idxs, path)

In [ ]:
ds, idxs = DatasetFormatter().from_similars(learn)